# Part 2 - Patient similarity networks

10. **Patient Similarity Network Construction**

11. **DNA Methylation Network Analysis** 

In [ ]:
# standard libraries
import os
import pickle

# scientific and data manipulation libraries
import numpy as np
import pandas as pd
from scipy import stats
from sklearn.feature_selection import mutual_info_regression
from astropy.stats import median_absolute_deviation
import mygene
import astropy

# graph and network libraries
import networkx as nx

# visualization libraries
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
import plotly.io as pio
from IPython.display import Image
from IPython.display import display

- Based on the same expression matrix we can create a patient similarity network.
- Transposing the matrix will switch the rows and columns,
- meaning that patients will become the columns instead of genes
- By doing this, you can compute the correlation (or similarity) between patients based on their gene expression profiles,
- and then create a network where nodes represent patients and edges represent similarities.

In [ ]:
# main data directories for the project

raw_data_dir = '/data/raw'
intermediate_data_dir = '/data/intermediate'

In [ ]:
# read in os.path.join(raw_data_dir,"expression_data_filtered.csv")
df_renamed = pd.read_csv(os.path.join(raw_data_dir,"expression_data_filtered.csv"),index_col=0)

In [ ]:
# Transpose the matrix
patient_gene_matrix = df_renamed.T

In [ ]:
patient_gene_matrix

In [ ]:
# Dictionary to store different correlation matrices
patient_correlation_matrices = {}

# Pearson correlation
patient_correlation_matrices['pearson'] = patient_gene_matrix.corr(method='pearson')

# patient_correlation_matrices['biweight_midcorrelation'] = abs_bicorr(patient_gene_matrix)

In [ ]:
def create_graph_from_correlation(correlation_matrix, threshold=0.8):
    """
    Creates a graph from a correlation matrix using a specified threshold.

    Parameters:
    correlation_matrix (pd.DataFrame): DataFrame containing the correlation matrix.
    threshold (float): Threshold for including edges based on correlation value.

    Returns:
    G (nx.Graph): Graph created from the correlation matrix.
    """
    G = nx.Graph()

    # Add nodes
    for node in correlation_matrix.columns:
        G.add_node(node)

    # Add edges with weights above the threshold
    for i in range(correlation_matrix.shape[0]):
        for j in range(i + 1, correlation_matrix.shape[1]):
            if i != j:  # Ignore the diagonal elements
                weight = correlation_matrix.iloc[i, j]
                if abs(weight) >= threshold:
                    G.add_edge(correlation_matrix.index[i], correlation_matrix.columns[j], weight=weight)

    return G


def clean_graph(G, degree_threshold=1, keep_largest_component=True):
    """
    Cleans the graph by performing several cleaning steps:
    - Removes unconnected nodes (isolates)
    - Removes self-loops
    - Removes nodes with a degree below a specified threshold
    - Keeps only the largest connected component (optional)

    Parameters:
    G (nx.Graph): The NetworkX graph to clean.
    degree_threshold (int): Minimum degree for nodes to keep.
    keep_largest_component (bool): Whether to keep only the largest connected component.

    Returns:
    G (nx.Graph): Cleaned graph.
    """
    G = G.copy()  # Work on a copy of the graph to avoid modifying the original graph

    # Remove self-loops
    G.remove_edges_from(nx.selfloop_edges(G))

    # Remove nodes with no edges (isolates)
    G.remove_nodes_from(list(nx.isolates(G)))

    # Remove nodes with degree below the threshold
    low_degree_nodes = [node for node, degree in dict(G.degree()).items() if degree < degree_threshold]
    G.remove_nodes_from(low_degree_nodes)

    # Keep only the largest connected component
    if keep_largest_component:
        largest_cc = max(nx.connected_components(G), key=len)
        G = G.subgraph(largest_cc).copy()

    return G

def print_graph_info(G):
    """
    Prints basic information about the graph.
    
    Parameters:
    G (nx.Graph): The NetworkX graph.
    """
    print(f"Number of nodes: {G.number_of_nodes()}")
    print(f"Number of edges: {G.number_of_edges()}")
    print("Sample nodes:", list(G.nodes)[:10])  # Print first 10 nodes as a sample
    print("Sample edges:", list(G.edges(data=True))[:10])  # Print first 10 edges as a sample
    
    # Check for self-loops
    self_loops = list(nx.selfloop_edges(G))
    if self_loops:
        print(f"Number of self-loops: {len(self_loops)}")
        print("Self-loops:", self_loops)
    else:
        print("No self-loops in the graph.")

def calculate_average_clustering(G):
    """
    Calculates and prints the average clustering coefficient of the graph.
    
    Parameters:
    G (nx.Graph): The NetworkX graph.
    """
    avg_clustering = nx.average_clustering(G)
    print(f"Average clustering coefficient: {avg_clustering}")


def knn_sparsification(graph, k):
    """
    Sparsifies the graph by keeping only the top-k edges with the highest weights for each node.

    Parameters:
    graph (nx.Graph): The original NetworkX graph.
    k (int): The number of nearest neighbors to keep for each node.

    Returns:
    nx.Graph: The sparsified graph.
    """
    graph_copy = graph.copy()
    sparsified_graph = nx.Graph()
    sparsified_graph.add_nodes_from(graph_copy.nodes(data=True))
    
    for node in graph_copy.nodes():
        edges = sorted(graph_copy.edges(node, data=True), key=lambda x: x[2].get('weight', 0), reverse=True)
        sparsified_graph.add_edges_from(edges[:k])
    
    return sparsified_graph

In [ ]:
patient_pearson_graph = create_graph_from_correlation(patient_correlation_matrices['pearson'], threshold=0.8)

In [ ]:
visualize_graph(patient_pearson_graph, title='Pearson Correlation Network (Threshold = 0.8)')

In [ ]:
# Clean the graph by removing unconnected nodes
patient_pearson_graph_pruned = clean_graph(patient_pearson_graph,
                                    degree_threshold=1,
                                    keep_largest_component=True)

In [ ]:
visualize_graph(patient_pearson_graph_pruned, title='Pearson Patient Correlation Network (Threshold = 0.8)')

In [ ]:
patient_pearson_graph_pruned_knn = knn_sparsification(patient_pearson_graph_pruned, k=10)

In [ ]:
print_graph_info(patient_pearson_graph_pruned)

In [ ]:
print_graph_info(patient_pearson_graph_pruned_knn)

In [ ]:
visualize_graph(patient_pearson_graph_pruned_knn, title='K-Nearest Neighbors (k=10) Patient Correlation Network')

In [ ]:
nx.write_gml(patient_pearson_graph_pruned_knn, os.path.join(intermediate_data_dir,'patient_coexpression_network_knn.gml'))

# DNA methylation based Patient Similarity Network

## Task

In the second task, we are preparing an additional network for the same patients, this time based on DNA methylation data.

In [ ]:
# Load the data using pickle
with open(os.path.join(data_dir,"ISMB_TCGA_DNAm.pkl") , 'rb') as file : 
    data = pd.read_pickle(file)

meth_data = data["datExpr"]
meth_data


In [ ]:
# load the data from the pickle file
with open(os.path.join(raw_data_dir,"ISMB_TCGA_GE.pkl"), 'rb') as file:
    GE_data = pickle.load(file)


In [ ]:
GE_data["datMeta"]["patient"].to_list()

1. Load the smoking-related DNA methylation data from a tab-separated values (TSV) file. (os.path.join(raw_data_dir,"smoking.tsv"))
2. Identify CpG sites that are commonly annotated in the smoking dataset
3. Filter the DNA methylation data to include only the common CpG sites identified in the previous step
4. Identify patients that are present in both the gene expression dataset and the methylation dataset.
5. Filter the methylation data to include only the common patients and common CpG sites.
6. Transpose the filtered methylation data matrix.

In [ ]:
smoking_df = pd.read_csv(os.path.join(raw_data_dir,"smoking.tsv") , delimiter='\t') #Tab seperated document from EWAS
common_annotated_cpgs = list(smoking_df['cpg'].value_counts()[smoking_df['cpg'].value_counts() > 10].index) 
cpgs = set(common_annotated_cpgs) & set(meth_data.columns)


common_patients = set(GE_data["datMeta"]["patient"].to_list()) & set(meth_data.index)


meth_data_filt = meth_data.loc[ list(common_patients) , list(cpgs)]

patient_meth_matrix = meth_data_filt.T


In [ ]:
patient_meth_matrix 


Calculate the pearson correlation of the data in a similar fashion.

In [ ]:
# Dictionary to store different correlation matrices
p_meth_correlation_matrices = {}

# Pearson correlation
p_meth_correlation_matrices['pearson'] = patient_meth_matrix.corr(method='pearson')


In [ ]:
p_meth_correlation_matrices['pearson']

In [ ]:
p_meth_pearson_graph = create_graph_from_correlation(p_meth_correlation_matrices['pearson'], threshold=0.8)
# Clean the graph by removing unconnected nodes
p_meth_pearson_graph_pruned = clean_graph(p_meth_pearson_graph,
                                    degree_threshold=1,
                                    keep_largest_component=True)

visualize_graph(p_meth_pearson_graph_pruned, title='Pearson Correlation Network (Threshold = 0.8)')


In [ ]:
p_meth_pearson_graph_pruned_knn = knn_sparsification(p_meth_pearson_graph_pruned, k=10)


In [ ]:
visualize_graph(p_meth_pearson_graph_pruned_knn, title='Pearson Correlation Network (Threshold = 0.8)')


In [ ]:
nx.write_gml(p_meth_pearson_graph_pruned_knn, os.path.join(data_dir,'patient_meth_network_knn.gml'))

In [ ]:
def save_graphs(correlation_matrices_dict, data_dir, matrix_names):
    if not os.path.exists(data_dir):
        os.makedirs(data_dir)
    
    # Define parameter combinations
    thresholds = [0.6, 0.7, 0.8, 0.9]
    k_values = [3, 5, 7, 10]
    degree_thresholds = [0, 1]
    keep_largest_component_options = [False, True]

    for matrix_name in matrix_names:
        if matrix_name not in correlation_matrices_dict:
            print(f"Matrix {matrix_name} not found in the provided dictionary. Skipping...")
            continue
        
        correlation_matrix = correlation_matrices_dict[matrix_name]
        
        for threshold in thresholds:
            # Create the initial graph from the correlation matrix
            initial_graph = create_graph_from_correlation(correlation_matrix, threshold=threshold)
            
            for degree_threshold in degree_thresholds:
                for keep_largest_component in keep_largest_component_options:
                    # Clean the graph
                    cleaned_graph = clean_graph(initial_graph, degree_threshold=degree_threshold, keep_largest_component=keep_largest_component)
                    
                    for k in k_values:
                        # Sparsify the graph using KNN
                        sparsified_graph = knn_sparsification(cleaned_graph, k)
                        
                        # Calculate the average clustering coefficient
                        avg_clustering = nx.average_clustering(sparsified_graph)
                        
                        # Construct the filename with relevant information
                        filename = f"{matrix_name}_network_threshold_{threshold}_k_{k}_degree_{degree_threshold}_largest_{keep_largest_component}_clustering_{avg_clustering:.2f}.gml"
                        filepath = os.path.join(data_dir, filename)
                        
                        # Save the graph to a GML file
                        nx.write_gml(sparsified_graph, filepath)
                        
                        # Print the details
                        print(f"Saved: {filepath}")

def main(correlation_matrices_dict, matrix_names):
    data_dir = '../../ismb_data/05072024/PSN-Met'  # Adjust the directory path as needed
    save_graphs(correlation_matrices_dict, data_dir, matrix_names)

if __name__ == "__main__":
    # correlation_matrices_dict = {
    #     'pearson': pd.DataFrame(),  # Replace with actual data
    #     'spearman': pd.DataFrame()  # Replace with actual data if needed
    # }
    matrix_names = ['pearson']  # Replace with the desired correlation matrix names
    main(p_meth_correlation_matrices, matrix_names)